In [2]:
import pandas as pd
import sys

sys.path.append('../scripts')
import problem

In [3]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

assert X_train.shape[1] == 284 + 331695